In [2]:
import pickle
with open('team_data/all_games_train.pkl', 'rb') as fp:
    all_games_train =  pickle.load(fp)
    
with open('team_data/all_games_test.pkl', 'rb') as fp:
    all_games_test =  pickle.load(fp)

In [3]:
import numpy as np
def data_to_vector(data, team_names):
    features = []
    labels = []
    for team in data:
        for game in data[team]:
            #extract features
            feature = [0] * 60
            #processing specific case for portland
            opp = game[0][0]
            if opp == 'portland':
                opp = 'portland trail'
            if opp[-1] == '*':
                opp = opp[:-1].strip()
            #computing feature vector
            if data[team][0][-1] == '1':
                feature[team_names.index(team)] = 1
                feature[30 + team_names.index(opp)] = 1
            else:
                feature[team_names.index(opp)] = 1
                feature[30 + team_names.index(team)] = 1
            features.append(feature)
            #extract labels
            labels.append(game[1])
    return (np.array(features), np.array(labels))

In [5]:
team_names = list(all_games_train.keys())
train_features, train_labels = data_to_vector(all_games_train, team_names)
test_features, test_labels = data_to_vector(all_games_test, team_names)

# svm

In [6]:
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score
def cal_accuracy(train_features, train_labels, test_features, test_labels):
    svm = LinearSVC(loss='hinge')
    svm.fit(train_features, train_labels)
    y_pred = svm.predict(test_features)
    return accuracy_score(test_labels, y_pred)

In [7]:
cal_accuracy(train_features, train_labels, test_features, test_labels)

0.612964180085442

# perceptron

In [17]:
from sklearn.datasets import load_digits
from sklearn.linear_model import Perceptron
clf = Perceptron()
clf.fit(train_features, train_labels)
y_pred = clf.predict(test_features)
accuracy_score(test_labels, y_pred)

/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.perceptron.Perceptron'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


0.5531547814656589

# neural network

In [15]:
#Model3 Fullly Connected Neural Network
import numpy as np
import torch
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.autograd import Variable
from torchvision import transforms

class Dataset(Dataset):
    """CIFAR-10 image dataset."""
    def __init__(self, X, y, transformations=None):
        self.len = len(X)           
        self.x_data = torch.from_numpy(X).float()
        self.y_data = torch.from_numpy(y).long()
    
    def __len__(self):
        return self.len

    def __getitem__(self, idx):
        return self.x_data[idx], self.y_data[idx]

class FeedForwardNN(nn.Module):
    """ 
        (1) Use self.fc1 as the variable name for your first fully connected layer
        (2) Use self.fc2 as the variable name for your second fully connected layer
    """
    def __init__(self):
        
        super(FeedForwardNN, self).__init__()
 
        self.fc1 = nn.Linear(60, 12)
        self.fc2 = nn.Linear(12, 2)


        

    def forward(self, x):
        out = F.tanh(self.fc1(x))
        out = F.sigmoid(self.fc2(out))

        
        return out
    

clf_nn = FeedForwardNN()

train_features = train_features.astype(int)
train_labels = train_labels.astype(int)
dataset = Dataset(train_features, train_labels)

train_loader = DataLoader(dataset = dataset,
                          batch_size = 100,
                          shuffle = True)


test_features = test_features.astype(int)
test_labels = test_labels.astype(int)
dataset = Dataset(test_features, test_labels)
test_loader = DataLoader(dataset = dataset,
                          batch_size = 100,
                        shuffle = True)

optimizer = optim.SGD(clf_nn.parameters(), lr = 0.001, momentum = 0.9, weight_decay = 5e-4)
criterion = torch.nn.CrossEntropyLoss()

max_epochs = 1000

accuracy = np.zeros((max_epochs))

for epoch in range(max_epochs):
    for i, data in enumerate(train_loader, 0):

        # Get inputs and labels from data loader 
        inputs, labels = data
        inputs, labels = Variable(inputs), Variable(labels)

        # Feed the input data into the network 
        y_pred = clf_nn(inputs)

        # Calculate the loss using predicted labels and ground truth labels

        loss = criterion(y_pred, labels)

        #print("epoch: ", epoch, "loss: ", loss.data[0])

        # zero gradient
        optimizer.zero_grad()
        
        # backpropogates to compute gradient
        loss.backward()

        # updates the weghts
        optimizer.step()

        # convert predicted laels into numpy
        y_pred_np = y_pred.data.numpy()

        # calculate the training accuracy of the current model

        label_np = labels.data.numpy().reshape(len(labels),1)

        correct = 0




correct = 0
total_size = 0
for i, data in enumerate(test_loader, 0):

    # Get inputs and labels from data loader 
    inputs, labels = data
    inputs, labels = Variable(inputs), Variable(labels)

    # Feed the input data into the network 
    y_pred = clf_nn(inputs)

    # convert predicted laels into numpy
    y_pred_np = y_pred.data.numpy()
    
    # calculate the training accuracy of the current model
    pred_np = y_pred_np  
    label_np = labels.data.numpy().reshape(len(labels),1)


    for j in range(y_pred_np.shape[0]):
        if np.argmax(pred_np[j,:]) == label_np[j,:]:
            correct += 1
    total_size += len(labels)
test_accuracy = float(correct) / float(total_size)

/anaconda3/lib/python3.6/site-packages/torch/nn/functional.py:995: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")
/anaconda3/lib/python3.6/site-packages/torch/nn/functional.py:1006: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


In [16]:
test_accuracy

0.6444298389746961